In [ ]:
#@title Оптимизация модели - убираем REI
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder  # LabelEncoder для преобразования меток

from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE  # Для балансировки данных
from xgboost import XGBClassifier  # Используем XGBoost
from collections import Counter

# Путь к папке с JSON файлами
output_dir = '/content/kata_elements_output'

# Список индексов ключевых точек MediaPipe, которые нас интересуют
required_indices = list(range(11, 17)) + list(range(23, 29))

# Функция для вычисления угла между тремя точками
def calculate_angle(a, b, c):
    """
    Вычисляет угол между тремя точками a, b, c.
    Точка b является центром угла.
    """
    # Векторы от точки b к точкам a и c
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    # Косинус угла через скалярное произведение
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Ограничение значения косинуса
    return np.degrees(angle)  # Преобразуем радианы в градусы

# Функция для загрузки данных из JSON файлов с добавлением углов
def load_data(output_dir):
    X = []  # Признаки (координаты точек + углы)
    y = []  # Метки классов (названия папок)

    for class_name in os.listdir(output_dir):
        if class_name.endswith('_poses_data.json'):
            class_label = class_name.split('_')[0]  # Извлекаем название класса
            json_path = os.path.join(output_dir, class_name)

            with open(json_path, 'r') as f:
                data = json.load(f)

            for pose_data in data:
                landmarks = pose_data['landmarks']
                # Создаем словарь для быстрого доступа к точкам
                landmark_dict = {landmark['id']: (landmark['x'], landmark['y'], landmark['z']) for landmark in landmarks}

                # Извлекаем координаты только для нужных точек
                features = []
                for i in required_indices:
                    if i in landmark_dict:
                        features.extend(landmark_dict[i])
                    else:
                        features.extend([0, 0, 0])  # Если точка не найдена, добавляем нули

                # Добавляем углы между точками
                angles = [
                    calculate_angle(landmark_dict.get(16, (0, 0, 0)), landmark_dict.get(14, (0, 0, 0)), landmark_dict.get(12, (0, 0, 0))),
                    calculate_angle(landmark_dict.get(14, (0, 0, 0)), landmark_dict.get(12, (0, 0, 0)), landmark_dict.get(24, (0, 0, 0))),
                    calculate_angle(landmark_dict.get(14, (0, 0, 0)), landmark_dict.get(12, (0, 0, 0)), landmark_dict.get(11, (0, 0, 0))),

                    calculate_angle(landmark_dict.get(15, (0, 0, 0)), landmark_dict.get(13, (0, 0, 0)), landmark_dict.get(11, (0, 0, 0))),
                    calculate_angle(landmark_dict.get(13, (0, 0, 0)), landmark_dict.get(11, (0, 0, 0)), landmark_dict.get(12, (0, 0, 0))),
                    calculate_angle(landmark_dict.get(13, (0, 0, 0)), landmark_dict.get(11, (0, 0, 0)), landmark_dict.get(23, (0, 0, 0))),

                    calculate_angle(landmark_dict.get(28, (0, 0, 0)), landmark_dict.get(26, (0, 0, 0)), landmark_dict.get(24, (0, 0, 0))),
                    calculate_angle(landmark_dict.get(26, (0, 0, 0)), landmark_dict.get(24, (0, 0, 0)), landmark_dict.get(12, (0, 0, 0))),
                    calculate_angle(landmark_dict.get(26, (0, 0, 0)), landmark_dict.get(24, (0, 0, 0)), landmark_dict.get(23, (0, 0, 0))),

                    calculate_angle(landmark_dict.get(27, (0, 0, 0)), landmark_dict.get(25, (0, 0, 0)), landmark_dict.get(23, (0, 0, 0))),
                    calculate_angle(landmark_dict.get(25, (0, 0, 0)), landmark_dict.get(23, (0, 0, 0)), landmark_dict.get(24, (0, 0, 0))),
                    calculate_angle(landmark_dict.get(25, (0, 0, 0)), landmark_dict.get(23, (0, 0, 0)), landmark_dict.get(11, (0, 0, 0)))
                ]
                features.extend(angles)

                X.append(features)
                y.append(class_label)

    return np.array(X), np.array(y)

# Загружаем данные
X, y = load_data(output_dir)

# Исключаем указанные классы
classes_to_exclude = [
    "l-chudan-gyaku-tsuki",
    "l-chudan-mae-geri",
    "l-gedan-barai",
    "l-jodan-age-uke",
    "r-chudan-mae-geri",
    "r-gedan-barai",
    "r-jodan-age-uke",
    "r-tsuru-ashi-dachi"
    "l-chudan-gyaku-uchi-uke",
    "l-chudan-shuto-uke",
    "r-chudan-shuto-uke",
    "l-turn-180",
    "l-turn-90",
    "l-zenkutsu-dachi-hanmi",
    "r-chudan-gyaku-uchi-uke",
    "r-chudan-nukite",
    "r-chudan-yoko-tettsui",
    "r-jodan-uraken",
    "r-jodan-yoko-keage",
    "r-morote-uke",
    "r-turn-180",
    "r-turn-90",
    "r-zenkutsu-dachi-hanmi",
    "l-chudan-gyaku-uchi-uke",
    "r-tsuru-ashi-dachi",
    "l-chudan-tettsui",
    "l-jodan-haiwan-uke",
    #"l-jodan-shuto-age-uke",
    #"l-step-45",
    "l-turn-270",
    "l-zenkutsu-dachi",
    "r-chudan-tettsui",
    "l-chudan-osae-uke",
    "r-chudan-gyaku-tsuki",
    "r-turn-135",
    "l-turn-45",
    "r-zenkutsu-dach",
    "l-chudan-yoko-tettsui",
    "r-zenkutsu-dachi",
    "r-jodan-haiwan-uke",
    #"r-step-fwd"
    "rei"
]
mask = ~np.isin(y, classes_to_exclude)
X = X[mask]
y = y[mask]

# Преобразуем метки классов в числовые значения с помощью LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Балансировка данных с помощью SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Создаём и обучаем модель (Random Forest) с учётом весов классов
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'  # Учёт весов классов
)
model.fit(X_train_resampled, y_train_resampled)

# Оцениваем модель на тестовой выборке
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Выводим отчёт о классификации с параметром zero_division
print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=label_encoder.classes_,  # Возвращаем оригинальные метки классов
    zero_division=0  # Устанавливаем значение метрик равным 0 при делении на ноль
))

In [ ]:
import pickle
# Сохраняем модель в формате .pkl
model_path = '/content/drive/MyDrive/Karate_2/dataset/random_forest_model_9_no_rei.pkl'
with open(model_path, 'wb') as file:
    pickle.dump(model, file)

print(f"Модель успешно сохранена в файл: {model_path}")

# Сохраняем LabelEncoder
label_encoder_path = '/content/drive/MyDrive/Karate_2/dataset/label_encoder_9_no_rei.pkl'
with open(label_encoder_path, 'wb') as file:
    pickle.dump(label_encoder, file)

# Сохраняем StandardScaler
scaler_path = '/content/drive/MyDrive/Karate_2/dataset/scaler_9_no_rei.pkl'
with open(scaler_path, 'wb') as file:
    pickle.dump(scaler, file)

print("LabelEncoder и StandardScaler успешно сохранены.")